In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# importing libraries
import random
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

#General Data/Plotting
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from tqdm.auto import tqdm 
import random

# Language
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re 
from collections import Counter
from string import punctuation

# Modeling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.metrics import precision_score, recall_score , f1_score, accuracy_score,confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

c:\Users\Nico\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_json('../Datasets/Cell_Phones_and_Accessories_5.json', lines=True)
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [4]:
df.drop(['reviewerID', 'asin', 'reviewerName', 'helpful','summary', 'unixReviewTime', 'reviewTime'], axis='columns', inplace=True)

In [5]:
df = df.rename(columns={"reviewText":"Review","overall": "Rating"})
df.columns

Index(['Review', 'Rating'], dtype='object')

In [6]:
df['review_len'] = [len(text.split()) for text in df.Review]
df.head()

,Review,Rating,review_len
0,They look good and stick good! I just don't li...,4,37
1,These stickers work like the review says they ...,5,32
2,These are awesome and make my phone look so st...,5,34
3,Item arrived in great time and was in perfect ...,4,51
4,"awesome! stays on, and looks great. can be use...",5,23


In [7]:
df = df[~(df['review_len'] < 20) & ~(df['review_len'] > 40)]

In [8]:
def convert_label(df) : 
    if df['Rating'] <= 3.0 : 
        rate = 0 # for Negative 
    else : 
        rate = 1 # for Positive
        
    return rate

In [9]:
df['Rating'] = df.apply(convert_label , axis = 1)

In [10]:
df.head()

,Review,Rating,review_len
0,They look good and stick good! I just don't li...,1,37
1,These stickers work like the review says they ...,1,32
2,These are awesome and make my phone look so st...,1,34
4,"awesome! stays on, and looks great. can be use...",1,23
5,These make using the home button easy. My daug...,0,23


In [11]:

# Set a random seed
random_seed = 42
random.seed(random_seed)
 
# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [12]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [15]:
textList = df['Review'].to_list()

In [16]:

# Input text

 
# Tokenize and encode text using batch_encode_plus
# The function returns a dictionary containing the token IDs and attention masks
encoding = tokenizer.batch_encode_plus(
    textList,                   # List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',      # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
)
 
input_ids = encoding['input_ids']  # Token IDs
# print input IDs
print(f"Input ID: {input_ids}")
attention_mask = encoding['attention_mask']  # Attention mask
# print attention mask
print(f"Attention mask: {attention_mask}")

Input ID: tensor([[ 101, 2027, 2298,  ...,    0,    0,    0],
        [ 101, 2122, 6293,  ...,    0,    0,    0],
        [ 101, 2122, 2024,  ...,    0,    0,    0],
        ...,
        [ 101, 2573, 2307,  ...,    0,    0,    0],
        [ 101, 2307, 4031,  ...,    0,    0,    0],
        [ 101, 1045, 2428,  ...,    0,    0,    0]])
Attention mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [17]:
# Generate embeddings using BERT model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    word_embeddings = outputs.last_hidden_state  # This contains the embeddings
 
# Output the shape of word embeddings
print(f"Shape of Word Embeddings: {word_embeddings.shape}")

RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 126396923904 bytes.